In [ ]:
# Necessary dependency for Credit File Review checklist
# needs an update for future
## Memorializing this as the easy way to lookup accounts


In [ ]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore


data = src.fetch_data.fetch_data()

# # # Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)
raw_data = cdutils.pkey_sqlite.add_ownership_key(raw_data)
raw_data = cdutils.pkey_sqlite.add_address_key(raw_data)

In [ ]:
househldacct = data['househldacct'].copy()

In [ ]:
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

In [ ]:
raw_data

In [ ]:
import cdutils.loans.calculations

In [ ]:
loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

In [ ]:
loan_category_df = loan_category_df[['acctnbr','Category']].copy()

In [ ]:
df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

In [ ]:
df

In [ ]:
df['Category'].unique()

In [ ]:
cml_loans = df[df['Category'].isin(["CRE","C&I","HOA"])].copy()

In [ ]:
pkey_grouping = cml_loans.groupby('portfolio_key')['Total Exposure'].sum().reset_index()

In [ ]:
pkey_grouping

In [ ]:
OUTPUT_PATH = Path(r"Z:\Data_Analytics\Libraries\cdutils\cdutils\acct_lookup\pkey_exposure.csv")
pkey_grouping.to_csv(OUTPUT_PATH)

In [ ]:
pkey_slice = df[['acctnbr','portfolio_key']].copy()
pkey_slice = pkey_slice.rename(columns={'portfolio_key':'HHNU'}).copy()
OUTPUT_PATH = 'hhnu.csv'
pkey_slice.to_csv(OUTPUT_PATH, index=False)